In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import numpy as np
import pandas as pd
import random
import os
import matplotlib.pyplot as plt
import pandas_datareader as web
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import * 
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import matplotlib.dates as dates
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.metrics import mean_absolute_error



from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Here we have a dataset consisting of the mean temeperature, humidity, wind speed and mean pressure in Delhi from the year 2013 to 2017.**
# **So we will try to predict the mean temperature for 2017 based on the data from 2013**

# Reading Training Data

In [25]:
data = pd.read_csv('../input/daily-climate-time-series-data/DailyDelhiClimateTrain.csv')

In [26]:
data.head()

In [27]:
data.tail()

In [28]:
#drop last row since it is already present in the test data
data = data.drop([1461])
data = data.drop(columns=['date']) #drop date column

In [29]:
#look for null values
data.isnull().sum()

In [30]:
data.head()

# Look for outliers using boxplots

In [31]:
plt.boxplot(data['meantemp'], vert=False)
plt.show()

In [32]:
plt.boxplot(data['humidity'], vert=False)
plt.show()

In [33]:
plt.boxplot(data['wind_speed'], vert=False)
plt.show()

This column has many outliers so we'll try to replace it with the median value

In [34]:
data['wind_speed'].describe()

In [35]:
median = data.loc[data['wind_speed']<15, 'wind_speed'].median()
data.loc[data['wind_speed'] < 1, 'wind_speed'] = np.nan
data.loc[data['wind_speed'] > 15, 'wind_speed'] = np.nan
data['wind_speed'].fillna(median,inplace=True)

In [36]:
#Boxplot after replcaing necessary outliers from wind_speed
plt.boxplot(data['wind_speed'], vert=False)
plt.show()

In [37]:
plt.boxplot(data['meanpressure'], vert=False)
plt.show()

In [38]:
data['meanpressure'].describe()

In [39]:
median1 = data.loc[data['meanpressure']<1050, 'meanpressure'].median()
data.loc[data['meanpressure'] > 1050 , 'meanpressure'] = np.nan #atmospheric pressure usually lies between 760 and 1050 atm
data.loc[data['meanpressure'] < 760 , 'meanpressure'] = np.nan
data['meanpressure'].fillna(median1,inplace=True)

In [40]:
#Boxplot after replacing necessary outliers from meanpressure
plt.boxplot(data['meanpressure'], vert=False)
plt.show()

In [41]:
plt.plot(data['meantemp'])
plt.xlabel('Day')
plt.ylabel('Mean Temperature')
plt.show()

# Standardize all columns

In [42]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler2=MinMaxScaler()
scaler3=MinMaxScaler()
scaler4=MinMaxScaler()

meantemp_scaled = scaler.fit_transform(data[['meantemp']])
humidity_scaled = scaler2.fit_transform(data[['humidity']])
windspeed_scaled = scaler3.fit_transform(data[['wind_speed']])
meanpressure_scaled = scaler4.fit_transform(data[['meanpressure']])

In [53]:
#Combining all scaled columns
X_scaled = np.concatenate((meantemp_scaled, humidity_scaled, windspeed_scaled, meanpressure_scaled), axis=1)
X_scaled.shape

# Creating batches of 30 days for training data

In [44]:
x=[]
for i in range(X_scaled.shape[0]-30):
    row = X_scaled[i:i+31]
    x.append(row)

x=np.array(x)
X_train = x[:,:-1] #1st 30 values will be considered as x values
Y_train = x[:,-1,0] #last value will be considered as our y value
X_train.shape ,Y_train.shape, X_train.ndim, Y_train.ndim

In [51]:
X_train = X_train.reshape(-1,4,30)

In [47]:
### DEFINE T2V LAYER ###

class T2V(Layer):
    
    def __init__(self, output_dim=None, **kwargs):
        self.output_dim = output_dim
        super(T2V, self).__init__(**kwargs)
        
    def build(self, input_shape):

        self.W = self.add_weight(name='W',
                                shape=(input_shape[-1], self.output_dim),
                                initializer='uniform',
                                trainable=True)

        self.P = self.add_weight(name='P',
                                shape=(input_shape[1], self.output_dim),
                                initializer='uniform',
                                trainable=True)

        self.w = self.add_weight(name='w',
                                shape=(input_shape[1], 1),
                                initializer='uniform',
                                trainable=True)

        self.p = self.add_weight(name='p',
                                shape=(input_shape[1], 1),
                                initializer='uniform',
                                trainable=True)

        super(T2V, self).build(input_shape)
        
    def call(self, x):
        
        original = self.w * x + self.p
        sin_trans = K.sin(K.dot(x, self.W) + self.P)
        
        return K.concatenate([sin_trans, original], -1)
    
    
    
## DEFINE MODEL STRUCTURES ###

def set_seed_TF2(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    

def T2V_NN(param, dim):
    
    inp = Input(shape=(dim,1))
    x = T2V(param['t2v_dim'])(inp)
    x = LSTM(param['unit'], activation=param['act'])(x)
    x = Dense(1)(x)
    
    m = Model(inp, x)
    m.compile(loss='mse', optimizer=Adam(lr=param['lr']))
    
    return m


def NN(param, dim):
    
    inp = Input(shape=(dim,1))
    x = LSTM(param['unit'], activation=param['act'])(inp)
    x = Dense(1)(x)
    
    m = Model(inp, x)
    m.compile(loss='mse', optimizer=Adam(lr=param['lr']))
    
    return m

### DEFINE PARAM GRID FOR HYPERPARM OPTIMIZATION ###

param_grid = {
    'unit': [64,32],
    't2v_dim': [128,64,16],
    'lr': [1e-2,1e-3], 
    'act': ['elu','relu'], 
    'epochs': 100, # 200,
    'batch_size': [128,512,1024]
}


base_param = {
    'unit': 64,
    't2v_dim': 64,
    'lr': 2e-3, 
    'act': 'relu', 
    'epochs': 100, # 200,
    'batch_size': 256
}

In [49]:
model = T2V_NN(param=base_param, dim=24)
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['accuracy'])

# model.fit(x_train, y_train, validation_split=0.2, callbacks=[es], shuffle=False)

# checkpointer = ModelCheckpoint(filepath = 'weights_best.hdf5', verbose = 1, save_best_only = True)
model.fit(X_train,Y_train,epochs=15,batch_size=64)
#  callbacks=[checkpointer])

In [58]:
def LSTM2_model():
    
    model = Sequential()
    model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1],1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    model.add(T2V_NN(param=base_param, dim=4))
    return model

model = LSTM2_model()
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['accuracy'])   ###

# checkpointer = ModelCheckpoint(filepath = 'weights_best.hdf5', verbose = 1, save_best_only = True)
his=model.fit(X_train,Y_train,epochs=20,batch_size=32)
# ,callbacks=[checkpointer])

# Creating LSTM Model

In [62]:
model=Sequential()
model.add(LSTM(100, input_shape=(4,30), return_sequences=True))
model.add(LSTM(100, return_sequences=True, recurrent_dropout=0.2))
model.add(LSTM(100, return_sequences=True, recurrent_dropout=0.2))
model.add(LSTM(100, return_sequences=False, recurrent_dropout=0.2))
model.add(Dense(1, activation='linear'))
model.add(T2V_NN(param=base_param, dim=24))

model.compile(optimizer='adam', loss='mean_squared_error' , metrics = ['accuracy'])

model.fit(X_train, Y_train, batch_size=32, epochs=50)

# Reading Testing Data

In [63]:
data_test = pd.read_csv('../input/daily-climate-time-series-data/DailyDelhiClimateTest.csv')

In [64]:
data_test.head()

In [65]:
data_test.tail()

In [66]:
#creating new dataframe which will have the last 30 rows of training data and the entire test data
new_data = pd.DataFrame()
new_data = (data.tail(30)).append(data_test)
new_data = new_data.drop(columns=['date'])

In [67]:
new_data.head()

In [68]:
new_data.shape

# Standardizing Test data

In [69]:
meantemp_scaled1 = scaler.fit_transform(new_data[['meantemp']])
humidity_scaled1 = scaler2.fit_transform(new_data[['humidity']])
windspeed_scaled1 = scaler3.fit_transform(new_data[['wind_speed']])
meanpressure_scaled1 = scaler4.fit_transform(new_data[['meanpressure']])

In [70]:
new_data_scaled = np.concatenate((meantemp_scaled1, humidity_scaled1, windspeed_scaled1, meanpressure_scaled1), axis=1)

# Creating batches of 30 days for Testing data

In [71]:
x1=[]
for i in range(new_data_scaled.shape[0]-30):
    row = new_data_scaled[i:i+31]
    x1.append(row)
    
x1=np.array(x1)
X_test = x1[:,:-1]
Y_test = x1[:,-1,0]
X_test.shape ,Y_test.shape, X_test.ndim, Y_test.ndim

In [72]:
X_test = X_test.reshape(-1,4,30)

# Making Predictions

In [75]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions) #converting the predictions to original scale again

Y_test = Y_test.reshape(-1,1) #convert Y_test to same shape as predictions
Y_test = scaler.inverse_transform(Y_test)

# Evaluating Predictions on Test data

In [76]:
#first 10 actual values
Y_test[:10]

In [77]:
#first 10 predicted values
predictions[:10]

In [78]:
#Coefficient of Determination
r2_score = r2_score(Y_test, predictions)
r2_score

In [79]:
#Mean Absolute Error 
mae = mean_absolute_error(Y_test, predictions)
mae

In [80]:
#Root Mean Square Error
mse = mean_squared_error(Y_test, predictions)
np.sqrt(mse)

In [81]:
plt.plot(Y_test, label='Actual Temperature', color='green')
plt.plot(predictions, label='Predicted Temperature', color='red')
plt.legend(loc='best')
plt.xlabel('Day')
plt.ylabel('Mean Temperature')
plt.show()